# Resolution Impact Analysis

This notebook analyzes the impact of image resolution and tiling on Mask R-CNN performance.


In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
# Load results
results_path = "reports/resolution_comparison.json"
with open(results_path, 'r') as f:
    data = json.load(f)

summary = data['summary']
per_image = data['per_image']

print(f"Loaded results for {summary['num_images']} images")


## 1. Line Plot: IoU Comparison Per Image


In [ ]:
image_ids = [r['image_id'] for r in per_image]
iou_512 = [r['512_iou'] for r in per_image]
iou_1024 = [r['1024_iou'] for r in per_image]
iou_tiled = [r['tiled_iou'] for r in per_image]

plt.figure(figsize=(14, 6))
plt.plot(image_ids, iou_512, 'o-', label='512×512', alpha=0.7)
plt.plot(image_ids, iou_1024, 's-', label='1024×512', alpha=0.7)
plt.plot(image_ids, iou_tiled, '^-', label='Tiled 1024×1024', alpha=0.7)
plt.xlabel('Image ID')
plt.ylabel('Mask IoU')
plt.title('Mask IoU Comparison: 512 vs 1024 vs Tiled (Per Image)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('reports/iou_comparison_line.png', dpi=150)
plt.show()


## 2. Bar Chart: Mean FDI Accuracy


In [ ]:
methods = ['512×512', '1024×512', 'Tiled']
fdi_accuracies = [
    summary['mean_512_fdi'],
    summary['mean_1024_fdi'],
    summary['mean_tiled_fdi']
]

plt.figure(figsize=(8, 6))
bars = plt.bar(methods, fdi_accuracies, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
plt.ylabel('Mean FDI Accuracy')
plt.title('Mean FDI Classification Accuracy by Method')
plt.ylim(0, 1.0)

# Add value labels on bars
for bar, acc in zip(bars, fdi_accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{acc:.3f}', ha='center', va='bottom')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('reports/fdi_accuracy_bar.png', dpi=150)
plt.show()


## 3. Error Distribution: IoU Histograms


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].hist(iou_512, bins=20, alpha=0.7, color='#3498db', edgecolor='black')
axes[0].set_title('512×512 IoU Distribution')
axes[0].set_xlabel('IoU')
axes[0].set_ylabel('Frequency')
axes[0].axvline(np.mean(iou_512), color='red', linestyle='--', label=f'Mean: {np.mean(iou_512):.3f}')
axes[0].legend()

axes[1].hist(iou_1024, bins=20, alpha=0.7, color='#2ecc71', edgecolor='black')
axes[1].set_title('1024×512 IoU Distribution')
axes[1].set_xlabel('IoU')
axes[1].set_ylabel('Frequency')
axes[1].axvline(np.mean(iou_1024), color='red', linestyle='--', label=f'Mean: {np.mean(iou_1024):.3f}')
axes[1].legend()

axes[2].hist(iou_tiled, bins=20, alpha=0.7, color='#e74c3c', edgecolor='black')
axes[2].set_title('Tiled IoU Distribution')
axes[2].set_xlabel('IoU')
axes[2].set_ylabel('Frequency')
axes[2].axvline(np.mean(iou_tiled), color='red', linestyle='--', label=f'Mean: {np.mean(iou_tiled):.3f}')
axes[2].legend()

plt.tight_layout()
plt.savefig('reports/error_distribution.png', dpi=150)
plt.show()


## 4. Box Plots: IoU Variance


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
bp = ax.boxplot([iou_512, iou_1024, iou_tiled], labels=['512×512', '1024×512', 'Tiled'],
                patch_artist=True)

# Color the boxes
colors = ['#3498db', '#2ecc71', '#e74c3c']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('Mask IoU')
ax.set_title('IoU Variance Comparison (Box Plots)')
ax.grid(True, alpha=0.3, axis='y')

# Add mean markers
means = [np.mean(iou_512), np.mean(iou_1024), np.mean(iou_tiled)]
for i, mean in enumerate(means):
    ax.plot(i+1, mean, 'rD', markersize=8, label='Mean' if i == 0 else '')

ax.legend()
plt.tight_layout()
plt.savefig('reports/iou_variance_boxplot.png', dpi=150)
plt.show()


## 5. Summary Statistics Table


In [ ]:
stats_data = {
    'Method': ['512×512', '1024×512', 'Tiled'],
    'Mean IoU': [summary['mean_512_iou'], summary['mean_1024_iou'], summary['mean_tiled_iou']],
    'Std IoU': [np.std(iou_512), np.std(iou_1024), np.std(iou_tiled)],
    'Mean mAP': [summary['mean_512_map'], summary['mean_1024_map'], summary['mean_tiled_map']],
    'Mean FDI Acc': [summary['mean_512_fdi'], summary['mean_1024_fdi'], summary['mean_tiled_fdi']],
    'Winner Count': [summary['winner_counts']['512'], summary['winner_counts']['1024'], summary['winner_counts']['tiled']]
}

df = pd.DataFrame(stats_data)
print(df.to_string(index=False))
